# Interactive chart using ipywidgets


Covid data from Johns Hopkins data repository. Using [ipywidgets](https://ipywidgets.readthedocs.io/en/stable) for interactive use.
* Display the rolling average for those countries with the highest number of deaths 
    * Change the number of days to set the rolling average (slider from 1 to 10)
    * Change the number of recent days to view (dropdown selection)



In [2]:
# import libraries
%matplotlib widget
import ipywidgets as widgets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load data for Deaths from COVID-19.

Load data into Pandas dataframes directly from Johns Hopkins data repository.



In [3]:
# Number of Death Cases - Global: time series data for deaths
deaths_path = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
deaths_df = pd.read_csv(deaths_path)

print(f"Shape of deaths = {deaths_df.shape}")

Shape of deaths = (274, 456)


In [4]:
def clean_covid_data(df):
    # 1. Drop unwanted columns
    df.drop(["Province/State", "Lat", "Long"], axis=1, inplace=True)

    # 2. Group by Country
    df = df.groupby("Country/Region").sum()

    # 3. Convert column headings to datetime
    df.columns = pd.to_datetime(df.columns)

    # 4. Remove the column name
    df.rename_axis(None, axis=0, inplace=True)

    return df

In [5]:
clean_deaths_df = clean_covid_data(deaths_df)

In [6]:
daily_deaths_df = clean_deaths_df.diff(axis=1)

---

In [7]:
def make_box_layout():
    return widgets.Layout(
        border="solid 1px black", margin="0px 10px 10px 0px", padding="5px 5px 5px 5px"
    )


class average_deaths(widgets.VBox):
    df = None
    
    def __init__(self, df):
        super().__init__()
        output = widgets.Output()
        self.df = df
        self.rolling_days = 7
        self.recent_days = 100

        with output:
            self.fig, self.ax = plt.subplots(
                figsize=(9, 5), facecolor=plt.cm.Blues(0.2)
            )
            self.fig.suptitle(
                "Deaths from COVID-19", fontsize="xx-large", fontweight="bold"
            )
            self.plot_axis()
            self.fig.autofmt_xdate()
            plt.tight_layout()
            self.fig.canvas.toolbar_position = "bottom"

        # define widgets
        int_slider = widgets.IntSlider(
            value=self.rolling_days,
            min=1,
            max=10,
            step=1,
        )
        recent_dropdown = widgets.Dropdown(
            options=[50, 100, 200, 300, 1000],
            value=self.recent_days,
            description="",
            disabled=False,
        )

        controls = widgets.HBox(
            [
                widgets.VBox(
                    [
                        widgets.Label("Rolling Average Days: "),
                        widgets.Label("Recent Number of Days: "),
                    ]
                ),
                widgets.VBox([int_slider, recent_dropdown]),
            ]
        )

        controls.layout = make_box_layout()
        out_box = widgets.Box([output])
        output.layout = make_box_layout()

        # observe changes
        int_slider.observe(self.update_rollout, "value")
        recent_dropdown.observe(self.update_recent, "value")

        # add the widgets to the view
        self.children = [controls, output]

    def update_rollout(self, change):
        """update the chart with changes in rolling frequency"""
        self.rolling_days = change.new
        self.plot_axis()
        self.fig.autofmt_xdate()
        self.fig.canvas.draw()

    def update_recent(self, change):
        """update recent number of days"""
        self.recent_days = change.new
        self.plot_axis()
        self.fig.autofmt_xdate()
        self.fig.canvas.draw()

    def line_color(self, change):
        self.line.set_color(change.new)

    def plot_axis(self):
        self.ax.clear()
        self.ax.set_title(
            f"{self.rolling_days}-day rolling average, for the last {self.recent_days} days"
        )
        self.ax.set_facecolor(plt.cm.Blues(0.2))
        rolling_df = self.df.rolling(self.rolling_days, axis=1).mean()
        top_countries = list(rolling_df.iloc[:, -1].sort_values(ascending=False).head(10).index)
        for i, country in enumerate(top_countries[:5]):
            self.ax.plot(
                self.df.columns[-self.recent_days :],
                rolling_df.loc[country].tail(self.recent_days),
                label=country,
                color=plt.cm.tab10(i),
            )
        self.ax.legend(
            bbox_to_anchor=(1, 0.8), loc="upper left", facecolor=plt.cm.Blues(0.1)
        )
        self.ax.set_xlabel("Date", fontsize="large", fontweight="bold")
        self.ax.set_ylabel("Number of Deaths", fontsize="large", fontweight="bold")
        self.ax.spines["right"].set_visible(False)
        self.ax.spines["top"].set_visible(False)


average_deaths(daily_deaths_df)

average_deaths(children=(HBox(children=(VBox(children=(Label(value='Rolling Average Days: '), Label(value='Rec…

---